                                                                Data Preparation

This notebook prepares the Chicago Crimes dataset for supervised machine learning.
All preprocessing steps are based on the insights obtained during the Exploratory Data Analysis (EDA).

The main objectives are:
- Clean the dataset
- Handle missing values
- Select relevant features
- Encode categorical variables
- Scale numerical features when needed
- Split the data into training and testing sets

In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from utils.preprocessing import (
    drop_redundant_columns,
    extract_datetime_features,
    fill_missing_values,
    remove_outliers_iqr
)


In [2]:
#Load Dataset
df = pd.read_csv("../data/city_of_chicago_crimes_2001_to_present.csv")
df.shape


(6747040, 22)

In [3]:
#Target Variable
df["Arrest"] = df["Arrest"].astype(int)


The target variable must be numerical in order to be used by supervised machine learning algorithms.
Here, Arrest is converted from a boolean variable to a binary variable:

1 → arrest

0 → no arrest

In [4]:
# Remove Duplicate Records
df = df.drop_duplicates(subset="ID")


Each crime record should appear only once in the dataset.
Removing duplicate records keeps the data consistent and prevents the model from learning the same event multiple times

In [5]:
# Drop Irrelevant and Redundant Columns
df = drop_redundant_columns(df)




***The dropped columns are:***

    ID, Case Number (identifiers)
    
    Block (high cardinality text)

    X Coordinate, Y Coordinate (duplicate of latitude and longitude)

    Location (string version of coordinates)

    Updated On (not related to the crime itself)

In [6]:
# drop columns with too many missing values
df = df.drop(columns=["Ward", "Community Area"])


From our notebook 1 the EDA we observed that :


    Ward has more than 600k missing values
    Community Area has more than 600k missing values
Filling these columns would introduce a lot of artificial data,
so we drop them completely.

In [7]:
df.head()


,Date,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,FBI Code,Year,Latitude,Longitude
0,03/18/2015 07:44:00 PM,041A,BATTERY,AGGRAVATED: HANDGUN,STREET,0,False,1111,11.0,04B,2015,41.891399,-87.744385
1,03/18/2015 11:00:00 PM,4625,OTHER OFFENSE,PAROLE VIOLATION,STREET,1,False,725,7.0,26,2015,41.773372,-87.665319
2,03/18/2015 10:45:00 PM,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,0,True,222,2.0,08B,2015,41.813861,-87.596643
3,03/18/2015 10:30:00 PM,0460,BATTERY,SIMPLE,APARTMENT,0,False,225,2.0,08B,2015,41.800802,-87.622619
4,03/18/2015 09:00:00 PM,031A,ROBBERY,ARMED: HANDGUN,SIDEWALK,0,False,1113,11.0,03,2015,41.878065,-87.743354


In [8]:
# Work on a representative subset to reduce computation time
df = df.sample(frac=0.2, random_state=42)

# Then extract datetime features
df = extract_datetime_features(df)


The Date column contains both date and time information.
We extract useful temporal features from it.

In [9]:
# handle remaining missing values
df = fill_missing_values(df)


After removing columns with too many missing values, some cells in the dataset are still empty.                     
                                                                                    
        For numerical columns, we replace missing values with the median (the middle value).                                                                                                                                                    
        For categorical columns, we replace missing values with the mode (the most common value).

In [10]:
# remove redundant crime encoding columns
df = df.drop(columns=["IUCR", "FBI Code"])


In our EDA notebook 1 we that :.

***IUCR , FBI Code and Primary Type***



all describe the same crime at different levels.

We already analyzed arrest outcomes using Primary Type, so we keep it and drop the others.

In [20]:
# Feature selection based on EDA findings

features = [
    "Primary Type",     # Crime category (EDA showed strong variation)
    "Domestic",         # Boolean context
    "District",         # Administrative area with different arrest rates
    "Latitude",         # Location (non-redundant)
    "Longitude",
    "Year",             # Temporal pattern observed in EDA
    "Month",
    "Day",
    "Hour",
    "DayOfWeek"
]

X = df[features]
y = df["Arrest"]


Feature selection is based on the exploratory analysis performed in Notebook 01.
Only variables that were shown to vary with arrest outcomes or provide essential
context (crime type, time, location, and administrative area) were retained.
Redundant, highly incomplete, or unexplored variables were excluded to keep the
model simple and interpretable.


In [21]:
# encode categorical variables
X = pd.get_dummies(X, drop_first=True)


Machine learning models require numerical inputs.
We apply one-hot encoding to categorical features.

In [22]:
# scale numerical features
scaler = StandardScaler()

num_features = [
    "Latitude",
    "Longitude",
    "Year",
    "Month",
    "Day",
    "Hour",
    "DayOfWeek"
]

X[num_features] = scaler.fit_transform(X[num_features])


c:\Users\CBG\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\CBG\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\CBG\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


We only scale continuous numerical variables.

In [27]:
# outlier detection
X = remove_outliers_iqr(X, num_features)
y = y.loc[X.index]


As required by the project, we apply a simple IQR-based outlier detection
on continuous numerical features.

In [29]:
# train / test split 
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

We split the data into training and testing sets.

In [16]:
X_train.shape, X_test.shape


NameError: name 'X_train' is not defined

In [30]:
df.head()


,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Year,Latitude,Longitude,Hour,Day,Month,DayOfWeek
6131905,THEFT,OVER $500,RESIDENCE,0,False,831,8.0,2008,41.762036,-87.703967,21,26,6,3
6130376,CRIMINAL DAMAGE,TO VEHICLE,STREET,0,False,1834,18.0,2008,41.890080,-87.627224,2,21,6,5
5408841,ASSAULT,SIMPLE,STREET,0,False,323,3.0,2006,41.765558,-87.605564,15,4,5,3
1853663,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,0,False,524,5.0,2008,41.671240,-87.651436,18,2,11,6
509033,THEFT,OVER $500,STREET,0,False,1832,18.0,2017,41.894328,-87.628143,2,3,6,5
